In [0]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&prompt=select_account&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&access_type=offline


Enter verification code: 4/HwF7bONIs4Yj45ON20zba8ME9L4S-b8RDOnl9H_3fBhyfS6CL1gEPBg
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [bergqvist@google.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import binarize, LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xgboost as xgb

In [0]:
!gsutil cp 'gs://aclima/cali_zip_bucket_export' ./

Copying gs://aclima/cali_zip_bucket_export...
\ [1 files][ 82.3 MiB/ 82.3 MiB]                                                
Operation completed over 1 objects/82.3 MiB.                                     


In [0]:
data = pd.read_csv('cali_zip_bucket_export')
data = data.sample(frac=1)
data = data.dropna()

In [0]:
data.head()

,hour,month,day,bc,no120,co2,no2,zipcode
24040,20,10,1,0.615336,10.510000,417.902008,8.072000,94501
391183,22,9,11,4.440056,20.000000,595.072998,8.686000,94002
65314,21,11,2,-0.533436,107.699997,524.174011,16.230000,94501
85907,19,5,3,0.592302,43.000001,480.303986,20.424999,94103
200780,17,9,6,0.649816,0.000000,413.049988,5.741000,94063


In [0]:
#plt.scatter(data['no2'].head())
#plt.show()

In [0]:
#print(data['bucket'].value_counts())

2    310850
4    310309
1    309863
3    306206
Name: bucket, dtype: int64


In [0]:
#data['month'] = data['month'].astype('int')
#data = pd.get_dummies(data, columns=['month'], dummy_na=True)
data['day'] = data['day'].astype('int')
data = pd.get_dummies(data, columns=['day'], dummy_na=True)
data['hour'] = data['hour'].astype('int')
data = pd.get_dummies(data, columns=['hour'], dummy_na=True)
#data['zipcode'] = data['zipcode'].astype('int')
#data = pd.get_dummies(data, columns=['zipcode'], dummy_na=True)

In [0]:
labels = data['no2']
#data = data.drop(columns=['bucket'])
data = data.drop(columns=['no2'])

In [0]:
data.head()

,month,bc,no120,co2,zipcode,day_1.0,day_2.0,day_3.0,day_4.0,day_5.0,...,day_nan,hour_16.0,hour_17.0,hour_18.0,hour_19.0,hour_20.0,hour_21.0,hour_22.0,hour_23.0,hour_nan
24040,10,0.615336,10.510000,417.902008,94501,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
391183,9,4.440056,20.000000,595.072998,94002,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
65314,11,-0.533436,107.699997,524.174011,94501,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
85907,5,0.592302,43.000001,480.303986,94103,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
200780,9,0.649816,0.000000,413.049988,94063,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [0]:
features = data

In [0]:
x,y = features,labels

x_train,x_test,y_train,y_test = train_test_split(x,y)
dmatrix = xgb.DMatrix(x_train, y_train)

bst = xgb.train({'objective': 'binary:logistic'}, dmatrix)

# Change to this for classification
#bst = xgb.train({'objective': 'binary:logistic'}, dmatrix)

y_pred = bst.predict(xgb.DMatrix(x_test))
model_filename = 'model.bst'
bst.save_model('model.bst')

NameError: ignored

In [0]:
#from sklearn.metrics import accuracy_score

xgb_diff = 0
num_test_samples = 100
for i,val in enumerate(y_pred[:num_test_samples]):
  xgb_diff += abs(val - y_test.iloc[i])
  print(val, 'predicted')
  print(y_test.iloc[i], 'actual')
  print('\n')

print(xgb_diff / num_test_samples)
#accuracy = accuracy_score(y_pred, y_test)

27.334116 predicted
26.145000457800002 actual


28.646444 predicted
33.7459983826 actual


28.166805 predicted
16.9309997559 actual


9.419685 predicted
10.114000320399999 actual


10.51365 predicted
9.28699970245 actual


11.061025 predicted
19.1599998474 actual


11.8735695 predicted
6.834000110630001 actual


3.8569462 predicted
3.9270000457800003 actual


11.420854 predicted
9.89900016785 actual


5.0768847 predicted
3.9119999408699995 actual


10.702138 predicted
9.081000328060002 actual


6.225499 predicted
4.27600002289 actual


3.6390274 predicted
1.9509999752 actual


2.082215 predicted
0.920000016689 actual


21.262205 predicted
24.0170001984 actual


8.82732 predicted
10.005000114400001 actual


18.146961 predicted
16.7569999695 actual


9.336089 predicted
10.494999885599999 actual


8.563932 predicted
2.3220000267 actual


4.3343515 predicted
3.4800000190699993 actual


2.9537103 predicted
1.38900005817 actual


7.666011 predicted
34.0419998169 actual


11.929635 predicted


In [0]:
xgb_diff = 0
num_test_samples = 100
for i,val in enumerate(y_pred[:num_test_samples]):
  xgb_diff += abs(val - y_test.iloc[i])
  print(val, 'predicted')
  print(y_test.iloc[i], 'actual')
  print('\n')

print(xgb_diff / num_test_samples)

27.334116 predicted
26.145000457800002 actual


28.646444 predicted
33.7459983826 actual


28.166805 predicted
16.9309997559 actual


9.419685 predicted
10.114000320399999 actual


10.51365 predicted
9.28699970245 actual


11.061025 predicted
19.1599998474 actual


11.8735695 predicted
6.834000110630001 actual


3.8569462 predicted
3.9270000457800003 actual


11.420854 predicted
9.89900016785 actual


5.0768847 predicted
3.9119999408699995 actual


10.702138 predicted
9.081000328060002 actual


6.225499 predicted
4.27600002289 actual


3.6390274 predicted
1.9509999752 actual


2.082215 predicted
0.920000016689 actual


21.262205 predicted
24.0170001984 actual


8.82732 predicted
10.005000114400001 actual


18.146961 predicted
16.7569999695 actual


9.336089 predicted
10.494999885599999 actual


8.563932 predicted
2.3220000267 actual


4.3343515 predicted
3.4800000190699993 actual


2.9537103 predicted
1.38900005817 actual


7.666011 predicted
34.0419998169 actual


11.929635 predicted


## Deploying to CMLE

In [0]:
!gcloud config set project bergqvist-sandbox

Updated property [core/project].


In [0]:
  MODEL_DIR="gs://aclima/model"
  VERSION_NAME="v4"
  MODEL_NAME="aclima"
  FRAMEWORK="XGBOOST"

In [0]:
!gsutil cp ./model.bst {MODEL_DIR}/model.bst

Copying file://./model.bst [Content-Type=application/octet-stream]...
/ [1 files][ 45.8 KiB/ 45.8 KiB]                                                
Operation completed over 1 objects/45.8 KiB.                                     


In [0]:
 !gcloud ml-engine versions create {VERSION_NAME} --model {MODEL_NAME} --origin {MODEL_DIR} --runtime-version=1.13 --framework {FRAMEWORK} --python-version=3.5

In [0]:
%%writefile predictions.json
[4, 0.411453, 3.0, 416.052002, 94501]

Writing predictions.json


In [0]:
predictions = !gcloud ml-engine predict --model={MODEL_NAME} --json-instances predictions.json --project=bergqvist-sandbox
print(predictions)

['\x1bERROR:\x1b (gcloud.ml-engine.predict) HTTP request failed. Response: {', '  "error": {', '    "code": 404,', '    "message": "Field: name Error: The model resource: \\"{MODEL_NAME}\\" was not found. Please create the Cloud ML model resource first by using \'gcloud ml-engine models create {MODEL_NAME}\'.",', '    "status": "NOT_FOUND",', '    "details": [', '      {', '        "@type": "type.googleapis.com/google.rpc.BadRequest",', '        "fieldViolations": [', '          {', '            "field": "name",', '            "description": "The model resource: \\"{MODEL_NAME}\\" was not found. Please create the Cloud ML model resource first by using \'gcloud ml-engine models create {MODEL_NAME}\'."', '          }', '        ]', '      }', '    ]', '  }', '}', '']


In [0]:
# Prediction docs (select XGBOOST): https://cloud.google.com/ml-engine/docs/scikit/quickstart